## Imports

In [ ]:
from plio.io import io_controlnetwork
from knoten.csm import create_csm
from scipy import sparse
import ale
import csmapi
import os
import numpy as np

import matplotlib.pyplot as plt

from knoten.bundle import *

In [ ]:
os.environ["ISISROOT"] = "/usgs/pkgs/isis3.7.0/install"
os.environ["ISIS3DATA"] = "/usgs/cpkgs/isis3/data"
from pysis import isis

## Inputs

In [ ]:
network = 'data_lak/hand_dense.net'
cubes = 'data_lak/cubes.lis'

## Functions

In [ ]:
def generate_sensors(cubes):
    isd_files = []
    sensors = {}
    for line in open(cubes):
        basename = os.path.splitext(os.path.basename(line.strip()))[0]
        isd = os.path.join('data_lak',basename+'.json')
        isd_files.append(isd)
        with open(isd, 'w+') as f:
            f.write(ale.loads(line.strip(), formatter='usgscsm'))

        sn = isis.getsn(from_=line.strip()).strip().decode('utf-8')
        sensors[sn] = create_csm(isd)
    return sensors

## Load Network
#### (Only Variables that vary cnet to cnet)

In [ ]:
cnet = io_controlnetwork.from_isis(network)
cnet

## Generate Sensors and Ground Points
#### For some reason, autoseed does not generate ground points for the control network, we have to calculate and repopulate the control network data frame

In [ ]:
sensors = generate_sensors(cubes)
cnet = compute_apriori_ground_points(cnet, sensors)

## Populate Jacobian

In [ ]:
parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()}
J, column_dict = compute_jacobian(cnet, sensors, parameters)
#rows are organized by just iterating over the control network

## Compute the Weight Matrix
#### According to the weighted Normal equation (J.TWJ), W needs to be a square matrix the size of (# of measures)x2. So it is the weight of the observations. In ISIS, the weight of the observations are an inverted function of the size of the pixels on the focal plane (resolution). However, in csm we do not have access to that information. 
#### For the time being, since we are working exclusively with CTX images we are going to set the weight matrix equal to the identity matrix -> all observations have the same weight.

In [ ]:
W = np.eye(260)

## Calculate Initial Sigma0

In [ ]:
def compute_residuals(network, sensors):
    """
    Compute the error in the observations by taking the difference between the
    ground points groundToImage projections and measure values.

    Parameters
    ----------
    network : DataFrame
              The control network as a dataframe generated by plio
    sensors : dict
             A dictionary that maps ISIS serial numbers to CSM sensors

    Returns
    -------
    V : np.array
       The control network dataframe with updated ground points
    """
    num_meas = len(network)
    V = np.zeros((num_meas, 2))

    for i in range(num_meas):
        row = network.iloc[i]
        serial = row["serialnumber"]
        ground_pt = row[["adjustedX", "adjustedY", "adjustedZ"]].values
        ground_pt = csmapi.EcefCoord(ground_pt[0], ground_pt[1], ground_pt[2])
        sensor = sensors[serial]
        img_coord = sensor.groundToImage(ground_pt)
        V[i,:] = [row['line'] - img_coord.line, row['sample'] - img_coord.samp]

    V = V.reshape(num_meas*2)
    return V

In [ ]:
V = compute_residuals(cnet, sensors)
print(V)
dof = J.shape[0] - J.shape[1]
sigma0 = np.sqrt((V.dot(W).dot(V))/dof)

print(sigma0)

## Bundle Iteration

In [ ]:
print(len([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, -0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1]))

In [ ]:
N = J.T.dot(W).dot(J)
C = J.T.dot(W).dot(V)
dX = np.linalg.inv(N).dot(C)
print(dX)

## Redundancy Number

In [ ]:
# redundancy for every measure
# vector will hold same order as the measures in the cnet df
# def compute_measure_redundancy
Qxx = np.linalg.inv(N)
Qvv = np.linalg.inv(W) - J.dot(Qxx).dot(J.T)
r = np.diagonal(Qvv.dot(W))

print(f'Minimum redundancy: {min(r)}')
print(f'Maximum redundancy: {max(r)}')
plt.boxplot(r)

## Data Snooping

In [ ]:
# For data snooping we need to calculate updated residuals
qvv = np.diagonal(Qvv)
sigma_vi = sigma0*np.sqrt(qvv)
wi = -V/sigma_vi

plt.figure()
plt.boxplot(wi)
plt.figure()
plt.hist(wi, bins=np.linspace(-4,5,50))

## Update Sensors

In [ ]:
# update the sensor partials
for sn, sensor in sensors.items():
    n_param = len(parameters[sn])
    for i in range(n_param):
        sensor.setParameterValue(i, dX[column_dict[sn]+i])
        
# update ground points
for _, row in cnet.iterrows():
    point_id = row['id']
    ground_pt = row[['adjustedX', 'adjustedY', 'adjustedZ']].values
    adj = dX[column_dict[point_id]:column_dict[point_id]+3] 
    cnet.loc[cnet.id == point_id, ["adjustedX", "adjustedY", "adjustedZ"]] = ground_pt + adj

In [ ]:
# check everything was actuall updated
print(cnet.iloc[0][['aprioriX', 'aprioriY', 'aprioriZ']])
print(cnet.iloc[0][['adjustedX', 'adjustedY', 'adjustedZ']])

sensor = sensors['MRO/CTX/1157902986:250']
print(sensor.getParameterValue(1))

In [ ]:
print(dof)

## Putting the whole bundle process into a loop

In [ ]:
cnet = io_controlnetwork.from_isis(network) # load in network
sensors = generate_sensors(cubes) # generate sensors
cnet = compute_apriori_ground_points(cnet, sensors) # calculate ground points

### INPUTS ###
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()} #all parameters
parameters = {sn: parameter[6:12] for sn, parameter in all_parameters.items()} #just solving for camera angles and angle velocity
W = np.eye(260)
tol = 1

##############

iteration = 0
V = compute_residuals(cnet, sensors)
dof = J.shape[0] - J.shape[1]
sigma0 = np.sqrt((V.dot(W).dot(V))/dof)
print(f'iteration {iteration}: sigma0 = {sigma0}')

# while abs(sigma0-old_sigma0) < tol:
iterations = 3
for i in range(iterations):   
    iteration += 1
    old_sigma0 = sigma0
    
    J, column_dict = compute_jacobian(cnet, sensors, parameters)    
    N = J.T.dot(W).dot(J) # calculate the normal equation
    C = J.T.dot(W).dot(V) 
    dX = np.linalg.inv(N).dot(C) #calculate change in camera parameters and ground points
    print(f'corrections: mean = {dX.mean()} min = {dX.min()} max = {dX.max()}')
    
    # update the sensor parameters
    for sn, sensor in sensors.items():
        n_param = len(parameters[sn])
        for i in range(n_param):
            old_param = sensor.getParameterValue(i)
            sensor.setParameterValue(i, old_param+dX[column_dict[sn]+i])

    # update ground points
    for _, row in cnet.iterrows():
        point_id = row['id']
        ground_pt = row[['adjustedX', 'adjustedY', 'adjustedZ']].values
        adj = dX[column_dict[point_id]:column_dict[point_id]+3] 
        cnet.loc[cnet.id == point_id, ["adjustedX", "adjustedY", "adjustedZ"]] = ground_pt + adj

    V = compute_residuals(cnet, sensors)
    dof = J.shape[0] - J.shape[1]
    sigma0 = np.sqrt((V.dot(W).dot(V))/dof)
    print(f'iteration {iteration}: sigma0 = {sigma0}')
    